# 개요
### 최동원 선수가 2015~2020년에 활동했더라면 연봉을 얼마를 받았을까? 

##### 우리 과제는 최동원의 롯데 시절 1983년부터 1988년의 데이터를 사용하도록 하겠습니다. 최동원의 당시 데이터와 함께 1983년부터 1988년 사이 투수들의 데이터(연봉포함)를 구하고, 2015년부터 2020년까지 프로야구의 투수들의 데이터를 구하시오.


In [3]:
import pandas as pd  
import numpy as np
import requests
from bs4 import BeautifulSoup

import re

import time
from tqdm import tqdm

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.by import By



/var/folders/3d/nr_splbx619_jz_ryc6wg6740000gn/T/ipykernel_6233/4011122596.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### 크롤링
#### 데이터 출저
###### - http://www.statiz.co.kr (투수 개인 기록 데이터)
###### - https://www.kaggle.com/datasets/mattop/korean-baseball-pitching-data-1982-2021 (팀 투수들의 평균 지표) 


In [ ]:
url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=1983&ye=1988&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=480&si=&cn='

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

records = []
birth_list = []

skip_row = 1
for tr in soup.find_all("tr")[1:]:
    if tr.find_all("th"):
        skip_row+=1
        continue
    tmp = []
    cnt=0
    flag_birth = True
    for text in tr.find_all("td"):
        if cnt == 3:
            cnt+=1
            continue
            
        add_text = '0' if (text.text.replace(' ', '') == '') else text.text

        try:
            tmp.append(float(add_text))
        except:
            tmp.append(add_text)
        cnt+=1
        
        #선수 생년월일 데이터
        try:
            if flag_birth:    
                birth_list.append(text.find("a")["href"][-10:])
                flag_birth = False
        except:
            pass
        
    if '질문/건의' in str(tmp[0]):
        break
    records.append(tmp)
    skip_row+=1

In [ ]:
print(records)

[[1.0, '선동열', '86해', 39.0, 19.0, 8.0, 22.0, 24.0, 6.0, 6.0, 0.0, 262.2, 38.0, 29.0, 972.0, 153.0, 0.0, 0.0, 2.0, 52.0, 5.0, 7.0, 214.0, 2.0, 3.0, 0.99, 1.16, 0.78, 307.1, 262.7, 14.89, 0.0], [2.0, '장명부', '83삼', 60.0, 36.0, 5.0, 44.0, 30.0, 16.0, 6.0, 0.0, 427.1, 138.0, 111.0, 1712.0, 388.0, 0.0, 0.0, 19.0, 106.0, 6.0, 16.0, 220.0, 1.0, 6.0, 2.34, 2.42, 1.16, 143.4, 138.5, 13.63, 0.0], [3.0, '선동열', '88해', 31.0, 9.0, 1.0, 12.0, 16.0, 5.0, 10.0, 0.0, 178.1, 29.0, 24.0, 674.0, 116.0, 0.0, 0.0, 3.0, 35.0, 0.0, 7.0, 200.0, 1.0, 0.0, 1.21, 1.26, 0.85, 316.2, 303.8, 11.79, 0.0], [4.0, '최동원', '86롯', 39.0, 17.0, 4.0, 21.0, 19.0, 14.0, 2.0, 0.0, 267.0, 60.0, 46.0, 1039.0, 204.0, 0.0, 0.0, 7.0, 55.0, 5.0, 6.0, 208.0, 0.0, 2.0, 1.55, 1.49, 0.97, 191.8, 200.0, 11.74, 0.0], [5.0, '김시진', '85삼', 47.0, 10.0, 2.0, 29.0, 25.0, 5.0, 10.0, 0.0, 269.2, 63.0, 60.0, 1064.0, 172.0, 0.0, 0.0, 8.0, 121.0, 3.0, 15.0, 201.0, 1.0, 10.0, 2.0, 2.59, 1.09, 177.3, 137.0, 10.52, 0.0], [6.0, '최동원', '85롯', 42.0, 14.0, 4.0,

### 데이터 프레임 만들기

In [ ]:
rank = []; name = []; team =[]; gp = []; cg = []; sho = []; gs = []; win = []; loss = [];
save = []; hold = []; inning = []; runs = []; earned_run = []; batter_faced = [];
hits = []; base2 = []; base3 = []; home_run = []; walk = []; intentional_walk = [];
hb = []; so = []; balk=[]; wild =[]; ERA = []; FIP = []; WHIP = []; ERAplus=[]; FIPplus=[]; WAR = []; WPA = []

status_list = [rank, name, team, gp, cg, sho, gs, win, loss, save, hold, inning, runs, earned_run, batter_faced, hits, base2, base3, home_run, walk, intentional_walk, hb, so, balk, wild, ERA, FIP, WHIP, ERAplus, FIPplus, WAR, WPA]

for record in records:
    for idx, status in enumerate(status_list):
        if idx == len(records)-1:
            break
        status.append(record[idx])
        
team_dict = {
    'M' : 'MBC Blue Dragons',
    'O' : 'OB Bears',
    '롯' : 'Lotte Giants',
    '빙' : 'Binggre Eagles',
    '삼' : 'Samsung Lions',
    '청' : 'Chungbo Pintos',
    '태' : 'Pacific Dolphins',
    '해' : 'Haitai Tigers',
    '해빙' : 'Haitai Tigers Binggre Eagles'
}

year = []
team_name = []

r = re.compile("[0-9]+")

for t in team:
    year.append('19'+r.findall(t)[0])
    team_name.append(team_dict[r.split(t)[1]])
    
birth_list = birth_list[:-1]

df_8388 = pd.DataFrame({
    'rank' : rank,
    'name' : name,
    'birth' : birth_list,
    'team' : team_name,
    'year' : year,
    'gp' : gp,
    'cg' : cg,
    'sho' : sho,
    'gs' : gs,
    'win' : win,
    'loss' : loss,
    'save' : save,
    'hold' : hold,
    'inning' : inning,
    'runs' : runs,
    'earned_run' : earned_run,
    'batter_faced' : batter_faced,
    'hits' : hits,
    'base2' : base2,
    'base3' : base3,
    'home_run' : home_run,
    'walk' : walk,
    'intentional_walk' : intentional_walk,
    'hb' : hb,
    'so' : so,
    'balk' : balk,
    'wild' : wild,
    'ERA' : ERA,
    'FIP' : FIP,
    'WHIP' : WHIP,
    'ERAplus' : ERAplus,
    'FIPplus' : FIPplus,
    'WAR' : WAR,
    'WPA' : WPA
})

df_8388.head()


,rank,name,birth,team,year,gp,cg,sho,gs,win,...,so,balk,wild,ERA,FIP,WHIP,ERAplus,FIPplus,WAR,WPA
0,1.0,선동열,1963-01-10,Haitai Tigers,1986,39.0,19.0,8.0,22.0,24.0,...,214.0,2.0,3.0,0.99,1.16,0.78,307.1,262.7,14.89,0.0
1,2.0,장명부,1950-12-27,Samsung Lions,1983,60.0,36.0,5.0,44.0,30.0,...,220.0,1.0,6.0,2.34,2.42,1.16,143.4,138.5,13.63,0.0
2,3.0,선동열,1963-01-10,Haitai Tigers,1988,31.0,9.0,1.0,12.0,16.0,...,200.0,1.0,0.0,1.21,1.26,0.85,316.2,303.8,11.79,0.0
3,4.0,최동원,1958-05-24,Lotte Giants,1986,39.0,17.0,4.0,21.0,19.0,...,208.0,0.0,2.0,1.55,1.49,0.97,191.8,200.0,11.74,0.0
4,5.0,김시진,1958-03-20,Samsung Lions,1985,47.0,10.0,2.0,29.0,25.0,...,201.0,1.0,10.0,2.00,2.59,1.09,177.3,137.0,10.52,0.0


### 구원 WAR 지표 가져오기 
### 구원 WAR지표는 단순히 url만 던져줘서 가져올 수 없었기 때문에 셀레니움을 사용하여 가져왔다.

In [ ]:
# 구원 투수 정보 가져오기 - 셀레니움

from selenium import webdriver
from selenium.webdriver.chrome.service import Service



chromedriver_path = '/Users/2suyeon/.cache/selenium/chromedriver/mac-arm64/121.0.6167.85/chromedriver'
service = Service(executable_path=chromedriver_path)

# Selenium WebDriver를 초기화
driver = webdriver.Chrome(service=service)

# 구체적인 웹사이트 URL 추가
driver.get(url)

relief_text = (driver.find_elements(By.ID, 'fixcol')[0].text)

driver.close()

relief_text = relief_text.replace('\n', ' ')
relief_text = relief_text.replace('순 이름 팀 정렬 구원WAR ', '')

relief_list = []
tmp = []
cnt = 0
for r_text in relief_text.split():
    tmp.append(r_text)
    cnt += 1
    try:
        if cnt == 4:
            float(r_text) #구원WAR 데이터가 없으면 에러 발생
            relief_list.append(tmp)
            tmp = []
            cnt = 0
    except:
        break
        
relief_year = []

r = re.compile("[0-9]+")

for t in relief_list:
    relief_year.append('19'+r.findall(t[2])[0])
    
    
idx_list = []
for i in range(len(relief_list)):
    idx_list.append(int(df_8388[(df_8388['name'] == relief_list[i][1]) & (df_8388['year'] == relief_year[i])].index.values[0]))
    

df_8388['relief_WAR'] = np.nan


for idx in idx_list:
    try:
        df_8388['relief_WAR'].iloc[idx] =float(relief_list[idx][3])
    except:
        pass
        
# 83-88 투수 데이터 저장
# df_8388.to_csv('1983-1988년 투수 데이터.csv')

#### 15 ~20 

In [ ]:
url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=2015&ye=2020&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=480&si=&cn='

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

records = []
birth_list = []

skip_row = 1
for tr in soup.find_all("tr")[1:]:
    if tr.find_all("th"):
        skip_row+=1
        continue
    tmp = []
    cnt=0
    flag_birth = True
    for text in tr.find_all("td"):
        if cnt == 3:
            cnt+=1
            continue
            
        add_text = '0' if (text.text.replace(' ', '') == '') else text.text

        try:
            tmp.append(float(add_text))
        except:
            tmp.append(add_text)
        cnt+=1
        
        #선수 생년월일 데이터
        try:
            if flag_birth:    
                birth_list.append(text.find("a")["href"][-10:])
                flag_birth = False
        except:
            pass
        
    if '질문/건의' in str(tmp[0]):
        break
    records.append(tmp)
    skip_row+=1

In [ ]:
rank = []; name = []; team =[]; gp = []; cg = []; sho = []; gs = []; win = []; loss = [];
save = []; hold = []; inning = []; runs = []; earned_run = []; batter_faced = [];
hits = []; base2 = []; base3 = []; home_run = []; walk = []; intentional_walk = [];
hb = []; so = []; balk=[]; wild =[]; ERA = []; FIP = []; WHIP = []; ERAplus=[]; FIPplus=[]; WAR = []; WPA = []

status_list = [rank, name, team, gp, cg, sho, gs, win, loss, save, hold, inning, runs, earned_run, batter_faced, hits, base2, base3, home_run, walk, intentional_walk, hb, so, balk, wild, ERA, FIP, WHIP, ERAplus, FIPplus, WAR, WPA]

for record in records:
    for idx, status in enumerate(status_list):
        if idx == len(records)-1:
            break
        status.append(record[idx])
        
team_dict = {
    'M' : 'MBC Blue Dragons',
    'O' : 'OB Bears',
    'S롯' : 'Lotte Giants',
    '롯' : 'Lotte Giants',
    'k롯' : 'Lotte Giants',
    '빙' : 'Binggre Eagles',
    '삼' : 'Samsung Lions',
    '청' : 'Chungbo Pintos',
    '태' : 'Pacific Dolphins',
    '해' : 'Haitai Tigers',
    '해빙' : 'Haitai Tigers Binggre Eagles',
    'K두' : 'Doosan Bears',
    '두' : 'Doosan Bears',
    'K' : 'Kia Tigers',
    'k' : 'KT Wiz',
    'L' : 'LG Twins',
    'Nk':'KT Wiz',
    '넥k' : 'KT Wiz',
    '넥' : 'Nexen Heroes',
    'S넥' :'Nexen Heroes',
    'S' :'SSG Landers',
    'LS':'SSG Landers',
    '키': 'Kiwoom Heroes',
    'N' : 'NC Dinos',
    '한' : 'Hanwha Eagles'
    
}



year = []
team_name = []

r = re.compile("[0-9]+")

for t in team:
    year.append('20'+r.findall(t)[0])
    team_name.append(team_dict[r.split(t)[1]])
    
birth_list = birth_list[:-1]



df_1520 = pd.DataFrame({
    'rank' : rank,
    'name' : name,
    'birth' : birth_list,
    'team' : team_name,
    'year' : year,
    'gp' : gp,
    'cg' : cg,
    'sho' : sho,
    'gs' : gs,
    'win' : win,
    'loss' : loss,
    'save' : save,
    'hold' : hold,
    'inning' : inning,
    'runs' : runs,
    'earned_run' : earned_run,
    'batter_faced' : batter_faced,
    'hits' : hits,
    'base2' : base2,
    'base3' : base3,
    'home_run' : home_run,
    'walk' : walk,
    'intentional_walk' : intentional_walk,
    'hb' : hb,
    'so' : so,
    'balk' : balk,
    'wild' : wild,
    'ERA' : ERA,
    'FIP' : FIP,
    'WHIP' : WHIP,
    'ERAplus' : ERAplus,
    'FIPplus' : FIPplus,
    'WAR' : WAR,
    'WPA' : WPA
})

df_1520.head()

,rank,name,birth,team,year,gp,cg,sho,gs,win,...,so,balk,wild,ERA,FIP,WHIP,ERAplus,FIPplus,WAR,WPA
0,1.0,알칸타라,1992-12-04,Doosan Bears,2020,31.0,1.0,0.0,31.0,20.0,...,182.0,1.0,3.0,2.54,3.10,1.03,182.6,149.7,8.31,4.35
1,2.0,양현종,1988-03-01,Kia Tigers,2015,32.0,1.0,1.0,31.0,15.0,...,157.0,0.0,6.0,2.44,4.57,1.24,202.1,108.5,8.14,3.94
2,3.0,스트레일리,1988-12-01,Lotte Giants,2020,31.0,0.0,0.0,31.0,15.0,...,205.0,0.0,4.0,2.50,3.01,1.02,195.1,159.0,7.53,3.84
3,4.0,양현종,1988-03-01,Kia Tigers,2019,29.0,2.0,2.0,29.0,16.0,...,163.0,0.0,1.0,2.29,2.63,1.07,183.1,160.3,7.35,3.81
4,5.0,브룩스,1990-04-27,Kia Tigers,2020,23.0,1.0,1.0,23.0,11.0,...,130.0,1.0,3.0,2.50,2.72,1.02,192.2,177.8,7.16,3.39


In [ ]:
pitcher_to_salary = []
year = []
salary = []


for idx in tqdm(range(len(df_1520['name']))):
    url = 'http://www.statiz.co.kr/player.php?opt=11&name=' + df_1520['name'][idx] + '&birth=' + df_1520['birth'][idx]
    
    response = requests.get(url)
    
    if response.status_code == 200:
        html = response.text
        soup_salary = BeautifulSoup(html, 'html.parser')
    else:
        print('!!! Error Request !!!')
    
    if [df_1520['name'][idx], df_1520['birth'][idx]] in pitcher_to_salary:
        continue
        
    pitcher_to_salary.append([df_1520['name'][idx], df_1520['birth'][idx]])
    
    tmp_year = []
    tmp_salary = []
    try:
        for s in soup_salary.find('table', class_ = 'table table-striped table-condensed').find_all("tr")[1:]:
            tmp_year.append(int(s.find_all("td")[0].text)) #년도
            try:
                tmp_salary.append(int(s.find_all("td")[1].text.replace(',',''))) #연봉
            except:
                tmp_salary.append(0)
    except:
        print("Error Pitcher : ", df_1520['name'][idx], '/', birth_list[idx])
        tmp_year.append(0)
        tmp_salary.append(0)
        
    year.append(tmp_year)
    salary.append(tmp_salary)
    
    time.sleep(0.3)
    


    


100%|██████████| 480/480 [12:14<00:00,  1.53s/it]


In [ ]:
df_1520.to_csv('save_df_1520.csv', encoding='utf-8')

In [ ]:
pitcher_to_salary

[['알칸타라', '1992-12-04'],
 ['양현종', '1988-03-01'],
 ['스트레일리', '1988-12-01'],
 ['브룩스', '1990-04-27'],
 ['린드블럼', '1987-06-15'],
 ['헥터', '1987-01-26'],
 ['김광현', '1988-07-22'],
 ['윌슨', '1989-09-25'],
 ['니퍼트', '1981-05-06'],
 ['켈리', '1988-10-14'],
 ['산체스', '1989-11-28'],
 ['브리검', '1988-02-10'],
 ['장원준', '1985-07-31'],
 ['해커', '1983-03-26'],
 ['루친스키', '1988-12-30'],
 ['요키시', '1989-07-29'],
 ['피어밴드', '1985-08-22'],
 ['밴헤켄', '1979-07-31'],
 ['신재영', '1989-11-18'],
 ['레일리', '1988-06-29'],
 ['허프', '1984-08-22'],
 ['소사', '1985-07-28'],
 ['우규민', '1985-01-21'],
 ['보우덴', '1986-09-09'],
 ['뷰캐넌', '1989-05-11'],
 ['박세웅', '1995-11-30'],
 ['구창모', '1997-02-17'],
 ['채드벨', '1989-02-28'],
 ['유희관', '1986-06-01'],
 ['스튜어트', '1986-09-28'],
 ['켈리', '1989-10-04'],
 ['피가로', '1984-07-07'],
 ['차우찬', '1987-05-31'],
 ['서폴드', '1990-01-16'],
 ['윤성환', '1981-10-08'],
 ['손승락', '1982-03-04'],
 ['후랭코프', '1988-08-27'],
 ['이용찬', '1989-01-02'],
 ['고우석', '1998-08-06'],
 ['윤석민', '1986-07-24'],
 ['문승원', '1989-11-28'],
 ['정우람', '1985-

In [ ]:
df_1520.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              480 non-null    float64
 1   name              480 non-null    object 
 2   birth             480 non-null    object 
 3   team              480 non-null    object 
 4   year              480 non-null    object 
 5   gp                480 non-null    float64
 6   cg                480 non-null    float64
 7   sho               480 non-null    float64
 8   gs                480 non-null    float64
 9   win               480 non-null    float64
 10  loss              480 non-null    float64
 11  save              480 non-null    float64
 12  hold              480 non-null    float64
 13  inning            480 non-null    float64
 14  runs              480 non-null    float64
 15  earned_run        480 non-null    float64
 16  batter_faced      480 non-null    float64
 1

In [146]:
add_to_salary_idx = []


salary_to_df = []
# 매칭 로직 수정 예시
for idx, (name, birth) in enumerate(zip(df_1520['name'], df_1520['birth'])):
    if idx < len(year) and idx < len(salary):  # 리스트 범위 내인지 확인
        for y, sal in zip(year[idx], salary[idx]):
        
            # 해당 연도와 이름, 생일이 일치하는 데이터프레임의 인덱스 찾기
            match_idx = df_1520[(df_1520['name'] == str(name)) & (df_1520['birth'] == str(birth)) & (df_1520['year'] == str(y))].index
            if not match_idx.empty:
                add_to_salary_idx.append(match_idx[0])
                salary_to_df.append(sal)
            else:
                print(f"매칭 실패: 이름={name}, 생일={birth}, 연도={y}")

                pass
    else:
        print(f"IndexError 방지: {idx}는 year 또는 salary 리스트의 범위를 벗어났습니다.")

df_1520 = df_1520.iloc[add_to_salary_idx]	# 연봉 데이터 매칭을 위해서 데이터 순서를 맞춰줌
df_1520['salary'] = salary_to_df			# 연봉 데이터 매칭
df_1520 = df_1520.sort_index()				# 매칭후 다시 원래 인덱스대로 정렬



매칭 실패: 이름=알칸타라, 생일=1992-12-04, 연도=2023
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2007
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2008
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2009
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2010
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2011
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2012
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2013
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2014
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2022
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2023
매칭 실패: 이름=스트레일리, 생일=1988-12-01, 연도=2021
매칭 실패: 이름=스트레일리, 생일=1988-12-01, 연도=2022
매칭 실패: 이름=스트레일리, 생일=1988-12-01, 연도=2023
매칭 실패: 이름=양현종, 생일=1988-03-01, 연도=2021
매칭 실패: 이름=브룩스, 생일=1990-04-27, 연도=2015
매칭 실패: 이름=브룩스, 생일=1990-04-27, 연도=2016
매칭 실패: 이름=브룩스, 생일=1990-04-27, 연도=2017
매칭 실패: 이름=브룩스, 생일=1990-04-27, 연도=2018
매칭 실패: 이름=브룩스, 생일=1990-04-27, 연도=2019
매칭 실패: 이름=린드블럼, 생일=1987-06-15, 연도=2007
매칭 실패: 이름=린드블럼, 생일=1987-06-15, 연도=2008
매칭 실패: 이름=린드블럼, 생일=1987-06-15, 연도=2009
매칭 실패: 이름=린드블럼, 생일=1987-06-15, 연도=2010
매칭 실패: 이름=린드블럼, 생일=1987-06-15, 연도=2011
매칭 실패: 이름=린드블럼, 생일=1987-06-15, 연도=2012

In [147]:
df_1520.head()

,rank,name,birth,team,year,gp,cg,sho,gs,win,...,balk,wild,ERA,FIP,WHIP,ERAplus,FIPplus,WAR,WPA,salary
0,1.0,알칸타라,1992-12-04,Doosan Bears,2020,31.0,1.0,0.0,31.0,20.0,...,1.0,3.0,2.54,3.10,1.03,182.6,149.7,8.31,4.35,45000
1,2.0,양현종,1988-03-01,Kia Tigers,2015,32.0,1.0,1.0,31.0,15.0,...,0.0,6.0,2.44,4.57,1.24,202.1,108.5,8.14,3.94,75000
1,2.0,양현종,1988-03-01,Kia Tigers,2015,32.0,1.0,1.0,31.0,15.0,...,0.0,6.0,2.44,4.57,1.24,202.1,108.5,8.14,3.94,45000
1,2.0,양현종,1988-03-01,Kia Tigers,2015,32.0,1.0,1.0,31.0,15.0,...,0.0,6.0,2.44,4.57,1.24,202.1,108.5,8.14,3.94,125000
2,3.0,스트레일리,1988-12-01,Lotte Giants,2020,31.0,0.0,0.0,31.0,15.0,...,0.0,4.0,2.50,3.01,1.02,195.1,159.0,7.53,3.84,50000


In [149]:
# 연봉 데이터 추가된 데이터 저장
df_1520.to_csv('2015-2020년 투수 데이터.csv', encoding='utf-8')

## 머신러닝

In [4]:
data_2015_2020 = pd.read_csv('/Users/2suyeon/Desktop/zerobase/미니프로젝트/ML_Project/Predictive Salary Analysis for Player Choi Dongwon/data/2015-2020년 투수 데이터.csv')
data_1983_1988 = pd.read_csv('/Users/2suyeon/Desktop/zerobase/미니프로젝트/ML_Project/Predictive Salary Analysis for Player Choi Dongwon/data/1983-1988년 투수 데이터.csv')

In [6]:
# 필요없는 컬럼 제거
data_1983_1988.drop(columns=['Unnamed: 0'], inplace=True)
data_2015_2020.drop(columns=['Unnamed: 0'], inplace=True)

# 결측치 확인
missing_values_1983_1988 = data_1983_1988.isnull().sum()
missing_values_2015_2020 = data_2015_2020.isnull().sum()

missing_values_1983_1988, missing_values_2015_2020


(rank                 0
 name                 0
 birth                0
 team                 0
 year                 0
 gp                   0
 cg                   0
 sho                  0
 gs                   0
 win                  0
 loss                 0
 save                 0
 hold                 0
 inning               0
 runs                 0
 earned_run           0
 batter_faced         0
 hits                 0
 base2                0
 base3                0
 home_run             0
 walk                 0
 intentional_walk     0
 hb                   0
 so                   0
 balk                 0
 wild                 0
 ERA                  0
 FIP                  0
 WHIP                 0
 ERAplus              0
 FIPplus              0
 WAR                  0
 WPA                  0
 relief_WAR          35
 dtype: int64,
 rank                0
 name                0
 birth               0
 team                0
 year                0
 gp                  0
 cg    

In [7]:
#결측치 확인
data_1983_1988.notnull

<bound method DataFrame.notnull of       rank name       birth           team  year    gp    cg  sho    gs   win  \
0      1.0  선동열  1963-01-10  Haitai Tigers  1986  39.0  19.0  8.0  22.0  24.0   
1      2.0  장명부  1950-12-27  Samsung Lions  1983  60.0  36.0  5.0  44.0  30.0   
2      3.0  선동열  1963-01-10  Haitai Tigers  1988  31.0   9.0  1.0  12.0  16.0   
3      4.0  최동원  1958-05-24   Lotte Giants  1986  39.0  17.0  4.0  21.0  19.0   
4      5.0  김시진  1958-03-20  Samsung Lions  1985  47.0  10.0  2.0  29.0  25.0   
..     ...  ...         ...            ...   ...   ...   ...  ...   ...   ...   
447  448.0  오문현  1953-02-13  Samsung Lions  1985  23.0   0.0  0.0   6.0   4.0   
448  449.0  노상수  1958-05-05   Lotte Giants  1986  16.0   1.0  0.0  10.0   3.0   
449  450.0  신태중  1955-06-27  Samsung Lions  1984  14.0   0.0  0.0   8.0   1.0   
450  451.0  엄평재  1960-10-24  Haitai Tigers  1983  13.0   0.0  0.0   1.0   0.0   
451  452.0  최상주  1962-08-07  Haitai Tigers  1985  32.0   0.0  0.0  10.0   

In [8]:
# 'relief_WAR' 컬럼에서 결측치를 갖고 있는 행만 제거 -> 선수를 제거 한다. 35명 의 데이터 제거
data_1983_1988_cleaned = data_1983_1988.dropna(subset=['relief_WAR'])

# 결측치 제거 후 데이터 확인
data_1983_1988_cleaned.shape, data_1983_1988_cleaned.isnull().sum().sum()


((417, 35), 0)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 연봉을 예측하기 위해 사용할 특성 선택
features = ['gp', 'cg', 'sho', 'gs', 'win', 'loss', 'save', 'hold', 'inning', 'so']
target = 'salary'

# 2015-2019년 데이터를 훈련 데이터로, 2020년 데이터를 테스트 데이터로 분할
train_data = data_2015_2020[data_2015_2020['year'] < 2020]
test_data = data_2015_2020[data_2015_2020['year'] == 2020]

X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# 선형 회귀 모델 학습
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = lr_model.predict(X_test)

# 모델 평가
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

rmse, r2


(34299.47299388287, 0.013988888697844093)

In [16]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb

# 랜덤 포레스트 모델 학습 및 예측
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# 그래디언트 부스팅 모델 학습 및 예측
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)



# 각 모델에 대한 평가 지표 계산
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_gb))
r2_gb = r2_score(y_test, y_pred_gb)



(rmse_rf, r2_rf), (rmse_gb, r2_gb)




((31502.97048572578, 0.16821721604331852),
 (30708.64947193756, 0.20963381920949298))

## 랜덤 포레스트 모델:
## RMSE (Root Mean Squared Error): 31,502.97
## R^2 Score: 0.168

## ----------------------------------

## 그래디언트 부스팅 모델:
## RMSE: 30,708.65
## R^2 Score: 0.210

In [17]:
# XGBoost 모델 학습 파라미터 조정  (간소화 하기 위해서)
xgb_model_simplified = xgb.XGBRegressor(
    objective ='reg:squarederror',
    seed=42,
    n_estimators=100,  # 트리의 수 조정
    max_depth=3,  # 트리의 최대 깊이 조정
    learning_rate=0.1  # 학습률 조정
)

xgb_model_simplified.fit(X_train, y_train)
y_pred_xgb_simplified = xgb_model_simplified.predict(X_test)

# 간소화된 XGBoost 모델에 대한 평가 지표 계산
rmse_xgb_simplified = np.sqrt(mean_squared_error(y_test, y_pred_xgb_simplified))
r2_xgb_simplified = r2_score(y_test, y_pred_xgb_simplified)

rmse_xgb_simplified, r2_xgb_simplified


(30488.77347015816, 0.22091145019231284)

### 앙상블 만들기


In [19]:
# 랜덤 포레스트와 그래디언트 부스팅 모델의 예측 결과를 평균내어 앙상블 모델 생성

# 이미 학습된 모델의 예측값을 사용
y_pred_rf = rf_model.predict(X_test)  # 랜덤 포레스트 예측
y_pred_gb = gb_model.predict(X_test)  # 그래디언트 부스팅 예측

# 예측 결과의 평균 계산
y_pred_ensemble = (y_pred_rf + y_pred_gb) / 2

# 앙상블 모델의 평가
rmse_ensemble = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))
r2_ensemble = r2_score(y_test, y_pred_ensemble)

rmse_ensemble, r2_ensemble



(30950.578702745206, 0.1971314205408754)

In [18]:
# 제공된 간소화된 XGBoost 모델의 성능 지표를 사용하여 앙상블 모델의 성능 계산을 수동으로 시뮬레이션합니다.

# 간소화된 XGBoost 모델의 예측 성능 지표
rmse_xgb_simplified_provided = 30488.77
r2_xgb_simplified_provided = 0.221

# 랜덤 포레스트, 그래디언트 부스팅, 간소화된 XGBoost 모델의 RMSE를 평균내어 계산
rmse_ensemble_provided_avg = (rmse_rf + rmse_gb + rmse_xgb_simplified_provided) / 3

# R^2 스코어는 평균내는 것이 적절하지 않을 수 있으므로, 제공된 정보를 바탕으로 직접 계산하지 않음
# 대신, 앙상블 모델이 개선될 수 있는 경향성을 논의

rmse_ensemble_provided_avg


30900.12998588778

In [20]:
#최동원

donghyun_row = data_1983_1988_cleaned[data_1983_1988_cleaned["name"] == "최동원"]

In [27]:
# 최동원님의 특성 데이터 선택
dongwon_features = data_1983_1988_cleaned[data_1983_1988_cleaned['name'] == '최동원'][features]

# 랜덤 포레스트 모델을 사용하여 최동원님의 연봉 예측
dongwon_salary_predictions = rf_model.predict(dongwon_features)


# 최동원님의 년도 데이터 다시 선택
dongwon_years = data_1983_1988_cleaned[data_1983_1988_cleaned['name'] == '최동원']['year']

# 년도와 예측값을 DataFrame으로 다시 변환 및 정렬
dongwon_predictions_df = pd.DataFrame({'Year': dongwon_years, 'Predicted Salary': dongwon_salary_predictions}).sort_values(by='Year')

# 정렬된 데이터 출력
for index, row in dongwon_predictions_df.iterrows():
    print(f'{int(row["Year"])}년 최동원 연봉 예측: {row["Predicted Salary"]:.2f}')






1983년 최동원 연봉 예측: 71766.99
1984년 최동원 연봉 예측: 63980.85
1985년 최동원 연봉 예측: 114958.87
1986년 최동원 연봉 예측: 113634.75
1987년 최동원 연봉 예측: 109937.67
1988년 최동원 연봉 예측: 16058.99
